In [1]:
import mne
import pandas as pd
import numpy as np
import glob
import pylab
import pandas as pd
import pickle
import matplotlib.pyplot as plt


stim_duration = .3
run_length = 4
first_run_file = 1
last_run_file = 4
S = 'S04'
bids_top_dir = '/System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/'
#proj_path = "/misc/data12/sjapee/Sebastian-OrientationImagery/Data/"
#raw_dir = proj_path + '20240911/' + S + '/'
#logfile_dir = raw_dir + 'sheets/extracted_data'
bids_dir = bids_top_dir + f'bids_dir/sub-{S}/ses-1/meg/sub-{S}_ses-1_task-OrientationImagery'

In [2]:
#*****************************#
### PARAMETERS ###
#*****************************#

l_freq                      = 0.1
h_freq                      = 100
notch                       = 60
notch_max                   = 240
pre_stim_time               = -.2
post_stim_time              = .6
std_deviations_above_below  = 4
output_resolution           = 200
trigger_channel             = 'UPPT001'


import glob

dsets = []
for i in range(1, 5, 1):
    ds = f'run-{i:02}'  # Format the run number with leading zeros
    pattern = f'{bids_dir}_{ds}_meg.ds'  # Adjust the pattern to match your file name structure
    matches = glob.glob(pattern)
    
    if matches:
        dsets.append(matches[0])
    else:
        print(f"No files found for pattern: {pattern}")

print(dsets)



['/System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/bids_dir/sub-S04/ses-1/meg/sub-S04_ses-1_task-OrientationImagery_run-01_meg.ds', '/System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/bids_dir/sub-S04/ses-1/meg/sub-S04_ses-1_task-OrientationImagery_run-02_meg.ds', '/System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/bids_dir/sub-S04/ses-1/meg/sub-S04_ses-1_task-OrientationImagery_run-03_meg.ds', '/System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/bids_dir/sub-S04/ses-1/meg/sub-S04_ses-1_task-OrientationImagery_run-04_meg.ds']


In [3]:
#New way of defining onsets not based on standard deviation but based on finding photodiode after the trigger
def defineOnsets(raw, events):
    log_statements = []
    raw.crop(0,raw.times[events[-1,0]]+10)
    pd_dat = raw[raw.ch_names.index('UADC016-2104')][0][0]
    print(np.mean(np.abs(pd_dat)))
    if np.mean(np.abs(pd_dat))>0.6: # this should be fine but may not be good if there are a looot of zeros
    # zero-centering & making everything positive
        pd_dat = np.abs(pd_dat-np.mean(pd_dat[0:100]))
        # scaling channel to 0-1
        pd_dat = (pd_dat-np.min(pd_dat)) / (np.max(pd_dat)-np.min(pd_dat))


        # using the event, we are now looking in the next 50ms of the photodiode channel (digital) when the signal goes above 0.5
        t_samples = int(0.05/(1/raw.info['sfreq']))
        threshold =  .5
        try:
            pd_events = [events[i,0]+np.where(pd_dat[events[i,0]:events[i,0]+t_samples]<threshold)[0][0] for i in range(len(events))]
        except:
            pd_events = [events[i,0]+np.where(pd_dat[events[i,0]:events[i,0]+t_samples]>threshold)[0][0] for i in range(len(events))]
        # when we found the events we calculate the trigger delay. 
        print(pd_events)
        pd_onsets = []
        pd_offsets = []

        for event in events:
            # Find onset
            onset_index = event[0] + np.argmax(pd_dat[event[0]:event[0] + t_samples] > threshold)
            print('test', onset_index)
            pd_onsets.append(onset_index)

            # Find offset
            pd_offset_index = onset_index + np.argmax(pd_dat[onset_index:] < threshold)
            pd_offsets.append(pd_offset_index)
        trigger_delay = 1000. * (pd_events-events[:, 0]) / raw.info['sfreq']
        # sometimes this channel records ON as UP and sometimes as DOWN. This if-loop makes sure we always find the onsets
        if np.mean(trigger_delay)==0:
            pd_events = [events[i,0]+np.where(pd_dat[events[i,0]:events[i,0]+t_samples]>threshold)[0][0] for i in range(len(events))]
            trigger_delay = 1000. * (pd_events-events[:, 0]) / raw.info['sfreq']
        log_statements.append(('Trigger delay removed (μ ± σ): %0.1f ± %0.1f ms')
            % (np.mean(trigger_delay), np.std(trigger_delay)))
        if np.mean(trigger_delay)>10:
            events[:,0] = events[:,0]+5
            log_statements.append(f'photo diode and parallel port triggers are too far apart. using parallel port trigger ')
        else:
            log_statements.append(f'photo diode used successfully ')
            events[:, 0] = pd_events
    else: 
    # if the pd channel didn't work, add 5 samples as a standard delay
        events[:,0] = events[:,0]+5
        log_statements.append(f'photo diode did not record anything. using parallel port trigger ')
    return raw,events,log_statements, pd_onsets, pd_offsets


def notch_filter(raw):
    #apply notch filter based on global parameters set at the top
    notches = np.arange(notch, notch_max+1, notch)
    print(notches)
    raw.notch_filter(notches, phase='zero-double', fir_design='firwin2')
    return raw

In [4]:
event_id = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

In [5]:
from mne.preprocessing import ICA


all_epochs = []
all_events = []
for i in range(len(dsets)):
    raw = mne.io.read_raw_ctf(dsets[i],system_clock='ignore',preload=True)
    events = mne.find_events(raw, stim_channel='UPPT001',min_duration = 0.002)
    print(events)
    #print('events', events)
    print(raw)
    #print('events2', events[-1])
    raw, events, log_statements, onsets, offsets = defineOnsets(raw, events)

    raw = notch_filter(raw)
    #ICA thing 
    epochs = mne.Epochs(raw, events, event_id=event_id,tmin=pre_stim_time, tmax=post_stim_time, preload=True, baseline=(None,0), picks = 'mag')
    print("Number of events:", len(events))
    print("Number of epochs:", len(epochs))
    print(np.nonzero(list(map(len, epochs.drop_log)))[0])
    [n for n, dl in enumerate(epochs.drop_log) if len(dl)]
    all_epochs.append(epochs)
    all_events.append(events)

print(len(all_events))

ds directory : /System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/bids_dir/sub-S04/ses-1/meg/sub-S04_ses-1_task-OrientationImagery_run-01_meg.ds
    res4 data read.
    hc data read.
    Separate EEG position data file not present.
    Quaternion matching (desired vs. transformed):
       1.22   64.10    0.00 mm <->    1.22   64.10   -0.00 mm (orig :  -38.06   56.72 -251.75 mm) diff =    0.000 mm
      -1.22  -64.10    0.00 mm <->   -1.22  -64.10   -0.00 mm (orig :   60.65  -24.77 -259.32 mm) diff =    0.000 mm
      75.01    0.00    0.00 mm <->   75.01   -0.00    0.00 mm (orig :   55.50   69.03 -226.26 mm) diff =    0.000 mm
    Coordinate transformations established.
    Polhemus data for 3 HPI coils added
    Device coordinate locations for 3 HPI coils added
    Measurement info composed.
Finding samples for /System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/bids_dir/sub-S04/ses-1/meg/sub-S04_ses-1_task-OrientationImagery_run-0

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.7s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    1.4s


Not setting metadata
264 matching events found
Removing 5 compensators from info because not all compensation channels were picked.
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 264 events and 961 original time points ...


[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:    2.3s


0 bad epochs dropped
Number of events: 264
Number of epochs: 264
[]
ds directory : /System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/bids_dir/sub-S04/ses-1/meg/sub-S04_ses-1_task-OrientationImagery_run-02_meg.ds
    res4 data read.
    hc data read.
    Separate EEG position data file not present.
    Quaternion matching (desired vs. transformed):
       1.81   64.10    0.00 mm <->    1.81   64.10    0.00 mm (orig :  -38.12   57.18 -251.93 mm) diff =    0.000 mm
      -1.81  -64.10    0.00 mm <->   -1.81  -64.10    0.00 mm (orig :   60.77  -24.11 -259.52 mm) diff =    0.000 mm
      76.84    0.00    0.00 mm <->   76.84    0.00   -0.00 mm (orig :   55.18   70.21 -222.54 mm) diff =    0.000 mm
    Coordinate transformations established.
    Polhemus data for 3 HPI coils added
    Device coordinate locations for 3 HPI coils added
    Measurement info composed.
Finding samples for /System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/bi

...

0 bad epochs dropped
Number of events: 264
Number of epochs: 264
[]
ds directory : /System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/bids_dir/sub-S04/ses-1/meg/sub-S04_ses-1_task-OrientationImagery_run-04_meg.ds
    res4 data read.
    hc data read.
    Separate EEG position data file not present.
    Quaternion matching (desired vs. transformed):
       1.95   64.51    0.00 mm <->    1.95   64.51    0.00 mm (orig :  -37.97   58.11 -252.00 mm) diff =    0.000 mm
      -1.95  -64.51    0.00 mm <->   -1.95  -64.51    0.00 mm (orig :   62.84  -22.08 -260.19 mm) diff =    0.000 mm
      78.93    0.00    0.00 mm <->   78.93    0.00   -0.00 mm (orig :   54.93   71.21 -216.17 mm) diff =    0.000 mm
    Coordinate transformations established.
    Polhemus data for 3 HPI coils added
    Device coordinate locations for 3 HPI coils added
    Measurement info composed.
Finding samples for /System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/bi

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.9s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    1.6s


Not setting metadata
264 matching events found
Removing 5 compensators from info because not all compensation channels were picked.
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 264 events and 961 original time points ...


[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:    2.4s


0 bad epochs dropped
Number of events: 264
Number of epochs: 264
[]
4


In [6]:
pattern = f'{bids_dir}*events.tsv'  # Find all event files in bids directory
matches = glob.glob(pattern)

matches.sort()
print(matches)

out = []

for event_files in matches[4:]:
    out.append(pd.read_csv(event_files,sep='\t'))
all_metadata = pd.concat(out).reset_index(drop=True)

all_metadata


['/System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/bids_dir/sub-S04/ses-1/meg/sub-S04_ses-1_task-OrientationImageryDynamic_run-01_events.tsv', '/System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/bids_dir/sub-S04/ses-1/meg/sub-S04_ses-1_task-OrientationImageryDynamic_run-02_events.tsv', '/System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/bids_dir/sub-S04/ses-1/meg/sub-S04_ses-1_task-OrientationImageryDynamic_run-03_events.tsv', '/System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/bids_dir/sub-S04/ses-1/meg/sub-S04_ses-1_task-OrientationImageryDynamic_run-04_events.tsv', '/System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/bids_dir/sub-S04/ses-1/meg/sub-S04_ses-1_task-OrientationImagery_run-01_events.tsv', '/System/Volumes/Data/misc/data12/sjapee/Sebastian-OrientationImagery/Data/Bids/bids_dir/sub-S04/ses-1/meg/sub-S04_ses-1_task-OrientationImagery_r

,onset,duration,trial_type,value,sample
0,81.295000,0.0,S01/Still/0292,14,97554
1,82.111667,0.0,S01/Still/0022,2,98534
2,82.928333,0.0,S01/Still/0337,16,99514
3,83.795000,0.0,S01/Still/0090,5,100554
4,84.645000,0.0,S01/Still/0225,11,101574
...,...,...,...,...,...
1051,247.305833,0.0,S01/Still/0225,11,296767
1052,248.172500,0.0,S01/Still/0315,15,297807
1053,249.039167,0.0,S01/Still/0022,2,298847
1054,249.889167,0.0,S01/Still/0180,9,299867


In [8]:
print(len(all_epochs[0]))

264


In [7]:
print(all_events[0])

[[ 97554      0     14]
 [ 98534      0      2]
 [ 99514      0     16]
 [100554      0      5]
 [101574      0     11]
 [102614      0      8]
 [103574      0     15]
 [104614      0      3]
 [105574      0      4]
 [106614      0      1]
 [107574      0     17]
 [108554      0     12]
 [109534      0     10]
 [110514      0     13]
 [111533      0      9]
 [112573      0      7]
 [113613      0      6]
 [114613      0     11]
 [115633      0      6]
 [116653      0     12]
 [117633      0     17]
 [118593      0     10]
 [119593      0      7]
 [120553      0     16]
 [121593      0     13]
 [122633      0      2]
 [123653      0      9]
 [126533      0     15]
 [127512      0      1]
 [128532      0      8]
 [129492      0     17]
 [130452      0      4]
 [131412      0     14]
 [132372      0      3]
 [133332      0      5]
 [134312      0      4]
 [135292      0      6]
 [136292      0     12]
 [137312      0     10]
 [138312      0     17]
 [139312      0     15]
 [140312      0 

In [11]:
print(all_epochs[0])

<Epochs |  264 events (all good), -0.2 – 0.6 s, baseline -0.2 – 0 s, ~520.9 MB, data loaded,
 '1': 15
 '2': 15
 '3': 15
 '4': 15
 '5': 15
 '6': 15
 '7': 15
 '8': 15
 '9': 15
 '10': 15
 and 7 more events ...>


In [9]:
dev_head_t_ref = all_epochs[0].info['dev_head_t']

for i in range(0, 4):
    all_epochs[i].info['dev_head_t'] = dev_head_t_ref

epochs_stacked = mne.concatenate_epochs(all_epochs)

/var/folders/9m/6b08zp213znc3k12g6gpjj9rqx2ly9/T/ipykernel_70835/987737906.py:6: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs_stacked = mne.concatenate_epochs(all_epochs)


Not setting metadata
1056 matching events found
Applying baseline correction (mode: mean)


In [10]:
epochs_stacked.metadata = all_metadata

epochs_stacked.metadata

Adding metadata with 5 columns


,onset,duration,trial_type,value,sample
0,81.295000,0.0,S01/Still/0292,14,97554
1,82.111667,0.0,S01/Still/0022,2,98534
2,82.928333,0.0,S01/Still/0337,16,99514
3,83.795000,0.0,S01/Still/0090,5,100554
4,84.645000,0.0,S01/Still/0225,11,101574
...,...,...,...,...,...
1051,247.305833,0.0,S01/Still/0225,11,296767
1052,248.172500,0.0,S01/Still/0315,15,297807
1053,249.039167,0.0,S01/Still/0022,2,298847
1054,249.889167,0.0,S01/Still/0180,9,299867


In [9]:
print(epochs_stacked.event_id)

{'1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, '11': 11, '12': 12, '13': 13, '14': 14, '15': 15, '16': 16, '17': 17}


In [10]:
print(len(epochs_stacked))

1056


In [37]:
print(epochs_stacked.event_id.keys())

dict_keys(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17'])


In [11]:
block_type = [k.split('/')[1] for k in epochs_stacked.metadata.trial_type]
condition = [k.split('/')[2] for k in epochs_stacked.metadata.trial_type]

n_runs = 4
n_trials = 264

epochs_stacked.metadata['run_nr'] = np.repeat(np.arange(n_runs),n_trials)
epochs_stacked.metadata['block_type'] = block_type
epochs_stacked.metadata['condition'] = condition

epochs_stacked.metadata

,onset,duration,trial_type,value,sample,run_nr,block_type,condition
0,81.295000,0.0,S01/Still/0292,14,97554,0,Still,0292
1,82.111667,0.0,S01/Still/0022,2,98534,0,Still,0022
2,82.928333,0.0,S01/Still/0337,16,99514,0,Still,0337
3,83.795000,0.0,S01/Still/0090,5,100554,0,Still,0090
4,84.645000,0.0,S01/Still/0225,11,101574,0,Still,0225
...,...,...,...,...,...,...,...,...
1051,247.305833,0.0,S01/Still/0225,11,296767,3,Still,0225
1052,248.172500,0.0,S01/Still/0315,15,297807,3,Still,0315
1053,249.039167,0.0,S01/Still/0022,2,298847,3,Still,0022
1054,249.889167,0.0,S01/Still/0180,9,299867,3,Still,0180


In [12]:
import os

# Construct the file path
file_name = f'sub-{S}_Still_preprocessed-epo.fif'
directory = f'{bids_top_dir}/derivatives/preprocessed/'

# Ensure the directory exists
os.makedirs(directory, exist_ok=True)

# Save the file
epochs_stacked.save(f'{directory}{file_name}', overwrite=True)


In [60]:
bids_dir

'/misc/data12/sjapee/Sebastian-OrientationImagery/Data/20240911/P1/bids_dir/sub-P1/ses-1/meg/'